## SHREC 2022 - PRIMITIVE FITTING

### Table of Contents

* [Preliminaries](#prelims)
* [Data Loading](#dl)
* [Dataset](#dset)
* [Baseline Model](#bmodel)
* [Losses](#losses)
* [Training Loop](#tloop)
* [Sources](#refs)

## Preliminaries <a class="anchor" id="prelims"></a>

The _"SHREC 2022: Fitting and recognition of simple geometric primitives on point clouds"_ track poses the challenge of recovering primitive shape parameters from 3d point clouds. The data consists of unordered points sampled from various primitives, possibly by adding some form of perturbation like noise, dropout, deformation etc. The set of all possible primitives and their parameters are as follows:

  * **Plane**, represented as its normal vector and a point sampled from the surface of the plane,
  * **Cylinder**, represented as its radius, rotation axis and a point sampled along said axis,
  * **Sphere**, represented as its radius and center,
  * **Cone**, represented as the rotational axis, half the aperture (the angle $\theta$ between the axis and any generatrix line) as well as a vertex.
  * **Torus**, represented as the major and minor radii, the rotational axis and the center.

The task is to build a framework that, given a point cloud, predicts the parameters of the primitive it was sampled from.

### Data Loading <a class="anchor" id="dl"></a>

In [1]:
import open3d as o3d
import os
import torch
import einops
from tqdm import tqdm

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


Having imported the necessary libary for displaying and manipulating the data as tensors, let us proceed to create appropriate functions for loading, conversions and visualizations.

In [49]:
def parse_point_cloud(fname):
    
    file = open(fname)
    points = []
    
    for line in file.readlines():
        pts = torch.Tensor(list(map(float, line.split(","))))
        points.append(pts)
    
    return einops.rearrange(points, "n d -> n d")

def tensor_to_o3d(pcloud):
    
    #sanity check
    assert pcloud.dim() == 2
    assert pcloud.shape[1] == 3
    
    #converting to numpy and removing device and associated gradients
    pcloud = pcloud.cpu().detach().numpy()
    
    #converting to open3d's data structures
    pcloud = o3d.utility.Vector3dVector(pcloud)
    pcloud = o3d.geometry.PointCloud(pcloud)
    
    return pcloud

#Displays a given point cloud using WebGL functionality
def show_point_cloud_o3d(pcloud):
    
    if isinstance(pcloud, torch.Tensor):
        pcloud = tensor_to_o3d(pcloud)
    
    o3d.visualization.draw_geometries([pcloud])

#Displays a given point cloud using open3d's JVisualizer
def show_point_cloud_jupyter(pcloud):
    
    if isinstance(pcloud, torch.Tensor):
        pcloud = tensor_to_o3d(pcloud)
    
    o3d.web_visualizer.draw(pcloud)

def show_point_cloud_with_normals_o3d(pcloud, radius=0.1, max_nn=30):
    
    if isinstance(pcloud, torch.Tensor):
        pcloud = tensor_to_o3d(pcloud)
    
    pcloud.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=radius, max_nn=max_nn))
    
    o3d.visualization.draw_geometries([pcloud], point_show_normal = True)
    
    
def category_indices(base_path, train_folder, gt_folder, train_prefix=None, gt_prefix=None, format=".txt"):
    
    '''
        This function assumes the following file structure:
        
        base_path
        -train_folder
            -train_prefix + {i} + .txt
        -gt_folder
            -gt_prefix + {i} + .txt
            
        And the following file contents:
        
        GT:
            a single integer corresponding to a category
        Train:
            N lines containing 3 comma-separated floats corresponding to point coordinates
    '''
    import os
    from tqdm import tqdm
    
    #
    train_prefix = train_prefix or train_folder 
    gt_prefix = gt_prefix or gt_folder 
    
    #
    train_path = os.path.join(base_path, train_folder)
    gt_path = os.path.join(base_path, gt_folder)
    
    #
    gt_file = lambda i : os.path.join(gt_path, gt_prefix + str(i) + format)
    train_file = lambda i : os.path.join(train_path, train_prefix + str(i) + format)
    
    #
    categories = {}
    
    #
    samples = os.listdir(train_path)
    N = len(samples)
    
    for i in tqdm(range(1, N+1)):
        with open(gt_file(i)) as GT:
            
            cat = GT.readline()[0]
            if cat not in categories.keys(): 
                categories[cat] = [i]
            else:
                categories[cat].append(i)
    
    save_path = os.path.join(base_path,"indices.txt");
    
    with open(save_path, "w") as F:
        for key in categories.keys():
            F.write(",".join(map(str, categories[key])) + "\n")

In [50]:
sample = 32004
#path = f"./data/point_clouds/pointCloud{sample}.txt"
path = f"C:\\Users\\vlassis\\Desktop\\phd\\datasets\\shrec2022\\training\\pointCloud\\pointCloud{sample}.txt"

base = "C:\\Users\\vlassis\\Desktop\\phd\\datasets\\shrec2022\\training"
category_indices(base, "pointCloud", "GTpointCloud")

#pcloud = parse_point_cloud(path)
#show_point_cloud_o3d(pcloud)
#show_point_cloud_with_normals_o3d(pcloud, radius=1)
#segment_plane_o3d(pcloud)
#show_point_cloud_jupyter(pcloud)

  2%|███▏                                                                                                                                                         | 925/46000 [00:00<00:04, 9205.06it/s]

adding category 1
adding category 2


  8%|████████████▌                                                                                                                                               | 3692/46000 [00:00<00:04, 9189.90it/s]

adding category 4
adding category 3


 12%|██████████████████▊                                                                                                                                         | 5531/46000 [00:00<00:04, 8451.32it/s]

adding category 5


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 46000/46000 [00:05<00:00, 8225.98it/s]


## Dataset <a class="anchor" id="dset"></a>

For a learning approach to this problem, we are going to need a dataset object and a dataloader that can be iterated. To do that we are going to be using the parser we created earlier as well as pytorch's `Dataset` class as a template. The data for this problem is unfortunately highly irregular, even if the task at hand seems easy at first glance.

There are 5 different types of primitives, each with its own set of parameters. The parameters themselves can have wildly different values which are had for a typical neural network to predict. Not only that, but the length of each set of parameters is different for each primitive, not allowing a "normal" representation of the labels.

The following dataset class will return the data and labels as a dictionary. The labels are their own dictionary, containing a string that specifies the type of primitive, its individual parameters as key-value pairs (for example `"radius" : 2.18`) and a torch tensor containing all of the parameters as one.

Naturally, a set of transformations are also included. A particularly important one is the unit-sphere normalization, which makes sure each point vector in the point cloud has a maximum length of one. This is accomplished by dividing the coordinates by the largest point vector's length. At this step it is important to save that normalization factor for later use.

In [99]:
class SHREC2022Dataset(torch.utils.data.Dataset):
    
    def __init__(self, path, train=True, valid=False, valid_split=0.2):
        
        self.path = os.path.join(path, "training" if train else "test")
        self.pc_prefix = "pointCloud"
        self.gt_prefix = "GTpointCloud"
        self.format = ".txt"
        self.valid = valid if train else False
        self.size = 0 if train else len(os.listdir(self.path))
        
        #check if an existing train-validation split matches the one given
        split_info_file = path + "training\\split_info.txt"
        self.t_savefile = path + "training\\train_split.txt"
        self.v_savefile = path + "training\\valid_split.txt"
        if os.path.exists(split_info_file):
            with open(split_info_file) as F:
                v, vsize, tsize = list(map(float, F.readline().split(',')))

                if v == valid_split:
                    print("Specified split already exists. Using the existing one.")
                    self.size = int(vsize if self.valid else tsize)
                    return
        
        if train:
            print("Creating a new train-validation split.")
            import random
            with open(path + "training\\indices.txt") as F, open(self.t_savefile, "w") as T,\
                open(self.v_savefile, "w") as V, open(split_info_file, "w") as I:

                lines = F.readlines()
                cat_sz = len(lines[0].split(","))
                train_sz = int(cat_sz * (1-valid_split))

                v_indices = []
                t_indices = []

                for line in lines:
                    line = list(map(int, line.split(",")))
                    random.shuffle(line)
                    v_indices = v_indices + line[train_sz:]
                    t_indices = t_indices + line[:train_sz]

                self.size = len(v_indices) if valid else len(t_indices)
                T.write("\n".join(map(str, t_indices)))
                V.write("\n".join(map(str, v_indices)))
                
                I.write(str(valid_split)+','+str(len(v_indices))+','+str(len(t_indices)))

        
    def parse_point_cloud(self, fname):
    
        file = open(fname)
        points = []

        for line in file.readlines():
            pts = torch.Tensor(list(map(float, line.split(","))))
            points.append(pts)

        file.close()
        
        return einops.rearrange(points, "n d -> n d")
    
    def parse_label(self, fname):
        
        file = open(fname)
        
        #assigning a distinct function to handle each type of primitive
        handlers ={
                   "1": self.parse_plane,
                   "2": self.parse_cylinder,
                   "3": self.parse_sphere,
                   "4": self.parse_cone,
                   "5": self.parse_torus
                  }
        
        #parsing the contents of the file. The first character corresponds to a specific type of primitive
        contents =  file.readlines()
        
        file.close()
        
        #handling the primitive and returning the label
        return handlers[contents[0][0]](contents)
    
    def parse_plane(self, lines):
        
        normal = torch.Tensor(list(map(float, lines[1:4])))
        vertex = torch.Tensor(list(map(float, lines[4:])))
        data = torch.Tensor(list(map(float, lines[1:])))
    
        return {"type": "plane", "class": 0, "vertex": vertex, "normal":normal, "data": data}
    
    def parse_cylinder(self, lines):
        
        radius = float(lines[1])
        axis = torch.Tensor(list(map(float, lines[2:5])))
        vertex = torch.Tensor(list(map(float, lines[5:])))
        data = torch.Tensor(list(map(float, lines[1:])))
    
        return {"type": "cylinder", "class": 1, "radius": radius, "axis": axis, "vertex": vertex, "data": data}
    
    def parse_sphere(self, lines):
        
        radius = float(lines[1])
        center = torch.Tensor(list(map(float, lines[2:])))
        data = torch.Tensor(list(map(float, lines[1:])))
    
        return {"type": "sphere", "class": 2, "radius": radius, "center": center, "data": data}
    
    def parse_cone(self, lines):
        
        angle = float(lines[1])
        axis = torch.Tensor(list(map(float, lines[2:5])))
        vertex = torch.Tensor(list(map(float, lines[5:])))
        data = torch.Tensor(list(map(float, lines[1:])))
    
        return {"type": "cone", "class": 3, "angle": angle, "axis": axis, "vertex": vertex, "data": data}
    
    def parse_torus(self, lines):
        
        major_radius = float(lines[1])
        minor_radius = float(lines[2])
        axis = torch.Tensor(list(map(float, lines[3:6])))
        center = torch.Tensor(list(map(float, lines[6:])))
        data = torch.Tensor(list(map(float, lines[1:])))
    
        return {"type": "torus", "class": 4, "major_radius": major_radius, "minor_radius": minor_radius, "axis": axis, "center": center, "data": data}
    
    def __getitem__(self, index):
        
        with open(self.v_savefile if self.valid else self.t_savefile, "r") as F:
            index = F.readlines()[index]
            index = int(index) if '\n' not in index else int(index[:-1])
         
        
        #assembling the file name for the data and labels
        pc_name = os.path.join(self.path, self.pc_prefix, self.pc_prefix + str(index) + self.format)
        gt_name = os.path.join(self.path, self.gt_prefix, self.gt_prefix + str(index) + self.format)
        
        #parsing the point cloud
        pcloud = self.parse_point_cloud(pc_name)
        label = self.parse_label(gt_name)
        
        data = {"x": pcloud, "y": label}
        
        return self.transform(data)
        
    def __len__(self):
        
        return self.size
    
    def transform(self, data):
        
        return self.unit_sphere_normalize(data)
        
    def unit_sphere_normalize(self, x):
        
        max_norm = (x["x"]*x["x"]).sum(-1).max().sqrt()
        x["x"] /= max_norm
        
        x["norm_factor"] = max_norm
        
        return x


def batch_collate_fn(batch_list):
    
    max_sz = int(torch.Tensor([item['x'].shape[0] for item in batch_list]).max().item())
    pad = torch.zeros(1, 3)
    
    print(max_sz)
    x = torch.stack([torch.cat((item['x'], pad.repeat((max_sz - item['x'].shape[0], 1))), dim=0) for item in batch_list])
    
    return {
           "x":   x,
           "y":   torch.Tensor([item['y']['class'] for item in batch_list]),
           #"z":   torch.stack([item['y']['data'] for item in batch_list]),
           "w":   [item['y'] for item in batch_list] 
    }
        
    
    

In [101]:
#dataset contains 9200 samples from each class
path = f"C:\\Users\\vlassis\\Desktop\\phd\\datasets\\shrec2022\\"
t_dataset = SHREC2022Dataset(path, train=True, valid=True, valid_split=0.2)
v_dataset = SHREC2022Dataset(path, train=True, valid=False, valid_split=0.2)

sample1 = t_dataset[0]
sample2 = v_dataset[0]
# show_point_cloud_o3d(sample1['x'])
# show_point_cloud_o3d(sample2['x'])
print(len(t_dataset))
print(len(v_dataset))

# path = f"C:\\Users\\vlassis\\Desktop\\phd\\shrec2022\\"
# plane_sample = 0
# cylinder_sample = 1000
# sphere_sample = 3000
# cone_sample = 2000
# torus_sample = 4000

# dataset = SHREC2022Dataset(path, train=True)
# batch = [dataset[0], dataset[1000], dataset[2000], dataset[3000], dataset[4000]]
# out = batch_collate_fn(batch)

# print(dataset[0])
# print(dataset[1000])
# print(dataset[2000])
# print(dataset[3000])
# print(dataset[4000])

Creating a new train-validation split.
Specified split already exists. Using the existing one.
9200
36800


### Baseline Model <a class="anchor" id="model"></a>

For the base model we will be testing out the PointNet we implemented in <a href="../../implementations/notebooks/pointnet.ipynb">this</a> notebook. Knowing the data has been affected by dropouts there are bound to be point clouds with considerably different cardinality. PointNet can consume point clouds of any number of input points, it is unnaffected by permutations and it is resistant to transformations such as rotation. This makes it suitable for a baseline model before we can move on to a more complex architecture.

Our implementation of PointNet outputs $k$ unnormalized scores which can be used for k-class classification. Since our data consists of 5 distinct primitives, each with its own set of parameters we are going to use PointNet as a feature extractor and feed the $k$ outputs to 5 regression heads, each one responsible for predicting a different set of parameters. Since we have no way of knowing in advance which primitive will be predicted, but we do know that each data sample contains exactly one, we have several choices:

  * Stack a 6th linear head on top of the model, with 5 outputs, responsible for classifying the input shape into one of the 5 categories. We can then predict parameters for all types of primitives, but only use the set corresponding to the classified primitive.
  
  * We can se all 5 of the regressors, and measure the distance between the input shape and each one of the predicted primitives. Then we choose the primitive that yielded the lowest distance as the correct one. 

In [6]:
%run ../../implementations/notebooks/pointnet.ipynb

regressor_outputs = {
    "plane": 6, "cylinder": 7, "cone": 7,
    "sphere": 4, "torus": 8
}

class Regressor(torch.nn.Module):
    
    def __init__(self, in_channels, out_channels):
        super(Regressor, self).__init__()
        
        self.reg = torch.nn.Sequential(
            torch.nn.Linear(in_channels, out_channels),
            torch.nn.LeakyReLU(),
            torch.nn.Linear(in_channels, in_channels)
        )
        
    def forward(self, x):
        
        return self.reg(x)

class BaselineModel(torch.nn.Module):
    
    def __init__(self, in_channels, feat_vector_dim):
        super(BaselineModel, self).__init__()
        
        self.pointnet = PointNet(in_channels, feat_vector_dim)
        
        self.reg_plane    = Regressor(feat_vector_dim, regressor_outputs["plane"])
        self.reg_cylinder = Regressor(feat_vector_dim, regressor_outputs["cylinder"])
        self.reg_cone     = Regressor(feat_vector_dim, regressor_outputs["cone"])
        self.reg_sphere   = Regressor(feat_vector_dim, regressor_outputs["sphere"])
        self.reg_torus    = Regressor(feat_vector_dim, regressor_outputs["torus"])
        self.shape_cls    = torch.nn.Linear(feat_vector_dim, len(regressor_outputs))
        
    def forward(self, x):
        
        feat_vec = self.pointnet(x)
        
        plane_params = self.reg_plane(feat_vec)
        cylinder_params = self.reg_cylinder(feat_vec)
        cone_params = self.reg_cone(feat_vec)
        sphere_params = self.reg_sphere(feat_vec)
        torus_params = self.reg_torus(feat_vec)
        class_preds = self.shape_cls(feat_vec)
        
        return class_preds, (plane_params, cylinder_params, cone_params, sphere_params, torus_params)

    

## Losses <a class="anchor" id="losses"></a>

Since finding an appropriate distance measure is not easy; an analytical one might not exist and a sample based one would have high computational requirements, the second implementation option is not feasible. Therefore, we choose to go with the former option, to include a classification head that chooses which primitive type the current example has been sampled from.

Simply classifying the primitive correctly is not enough however, as the output primitive parameters must have sensible values as well. We must therefore employ specifically tailored regularization losses that tend to each type of parameter and force it to have approximately "proper" values. 

Let's assume, for instance, that we have correctly classified a primitive as "plane". Then the angle between the predicted and the actual normal of the plane must be small, and the distance of the predicted vertex from the actual plane must be low. This can be as easy as using a mean squared error criterion, but maybe more specific distance measures will yield better results. Both approaches are worth attempting.

In [7]:
#cross entropy criterion for classification
cls_loss = torch.nn.CrossEntropyLoss()

#------------------APROACH 1----------------------
#mean squared error criterion for the rest
param_loss = torch.nn.MSELoss()

#------------------APROACH 2----------------------
#axis loss
def axis_loss(ax1, ax2):
    return torch.dot(ax1, ax2)

#
B, num_epochs = 2, 50
path = f"C:\\Users\\vlassis\\Desktop\\phd\\datasets\\shrec2022\\"
dataset = SHREC2022Dataset(path, train=True)
dataloader = torch.utils.data.DataLoader(dataset, batch_size = B, collate_fn = batch_collate_fn, shuffle = True, drop_last = False)

model = BaselineModel(3, 256).cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

## Training Loop <a class="anchor" id="tloop"></a>

In [8]:
for i in range(num_epochs):
    
    m_loss = 0
    for batch in dataloader:
        
        x = batch["x"].cuda()
        y = batch["y"].cuda()
        
        optimizer.zero_grad()
        
        pred, _ = model(x.permute(0,2,1))
        
        loss = cls_loss(pred, y)
        
        loss.backward()
        
        optimizer.step()
        
        m_loss += loss.item() / B
        
    m_loss /= len(dataloader)
    
    print(f"epoch {i} - loss: {m_loss}")

8099


TypeError: linear(): argument 'input' (position 1) must be Tensor, not tuple

### Sources <a class="anchor" id="refs"></a>